# Autonomy POC Camera Selection

Author: Paul Barsic  
paul.barsic@terraclear.com  
13 December 2021

### Summary

This notebook contains some good methods for comparing sensor resolution. The purpose of this comparison is to inform Terraclear's camera choices for the Autonomy POC work that needs to be done in January 2022. I think that the best camera available is the Luxonis Oak-D POE camera, which combines a stereo sensor with a high resolution RGB sensor with a sufficiently small GSD, in an IP-67 package.

In [1]:
import numpy as np

In [2]:
def dfov(width_px, height_px, pitch_um, f_mm):
    half_diag_px = 0.5 * np.sqrt(width_px ** 2 + height_px ** 2)
    half_diag_mm = half_diag_px * pitch_um * 1e-3
    theta = 2 * np.arctan(half_diag_mm / f_mm)

    return theta


def hfov(width_px, height_px, pitch_um, f_mm):
    return dfov(width_px, 0, pitch_um, f_mm)


def vfov(width_px, height_px, pitch_um, f_mm):
    return dfov(0, height_px, pitch_um, f_mm)


def gsd(pitch_um, f_mm, distance):
    return distance * np.tan(hfov(1, 1, pitch_um, f_mm))


def print_summary(identifier, width_px, height_px, pitch_um, f_mm, distance_m):
    theta_d = dfov(width_px, height_px, pitch_um, f_mm)
    theta_h = hfov(width_px, height_px, pitch_um, f_mm) * 180.0 / np.pi
    theta_v = vfov(width_px, height_px, pitch_um, f_mm) * 180.0 / np.pi
    gsd_mm = gsd(pitch_um, f_mm, distance_m * 1e3)
    print(
        f"{identifier}\n"
        f" focal length   = {f_mm:.2f} mm\n"
        f" FOV   Diagonal = {theta_d * 180. / np.pi:.1f} degrees\n"
        f"     Horizontal = {theta_h:.1f} degrees\n"
        f"       Vertical = {theta_v:.1f} degrees\n"
        f"  GSD at {distance_m:.2f} m = {gsd_mm:.1f} mmm\n"
    )


def compute_pitch_um(dfov_rad, width_px, height_px, f_mm):
    half_diag_px = 0.5 * np.sqrt(width_px ** 2 + height_px ** 2)
    pitch_um = np.tan(dfov_rad) * f_mm / half_diag_px * 1e3
    return pitch_um


def compute_focal_mm(dfov_rad, width_px, height_px, pitch_um):
    half_diag_mm = 0.5 * np.sqrt(width_px ** 2 + height_px ** 2) * pitch_um * 1e-3
    f_mm = half_diag_mm / np.tan(dfov_rad * 0.5)
    return f_mm


# Camera orientation
The next box defines the model camera mounting parameters: height above ground, and angle that the camera is tilted down from the horizon.

In [11]:
camera_height_m = 2.0
camera_declination_degrees = 30
distance_m = camera_height_m / np.sin(camera_declination_degrees * np.pi / 180.)
print(f"Camera center projects to a point on the ground {distance_m:0.2f} m away.")

Camera center projects to a point on the ground 4.00 m away.


## Zed 2i 2mm

In [9]:
focal_mm = 2.2
width_pixels = 2208
height_pixels = 1242
pitch_um = 2.8

print_summary("Zed 2i, 2mm lens option", width_pixels, height_pixels, pitch_um, focal_mm, distance_m)

Zed 2i, 2mm lens option
 focal length   = 2.20 mm
 FOV   Diagonal = 116.4 degrees
     Horizontal = 109.1 degrees
       Vertical = 76.6 degrees
  GSD at 4.00 m = 5.1 mmm



## Zed 2i 4mm

In [10]:
focal_mm = 4.0
width_pixels = 2208
height_pixels = 1242
pitch_um = 2.8

print_summary("Zed 2i, 4mm lens option", width_pixels, height_pixels, pitch_um, focal_mm, distance_m)

Zed 2i, 4mm lens option
 focal length   = 4.00 mm
 FOV   Diagonal = 83.1 degrees
     Horizontal = 75.4 degrees
       Vertical = 47.0 degrees
  GSD at 4.00 m = 2.8 mmm



## Lucid TRI230c

In [6]:

focal_mm = 12.0
width_pixels = 1920
height_pixels = 1200
pitch_um = 3.45

print_summary("Lucid TRI230c", width_pixels, height_pixels, pitch_um, focal_mm, distance_m)

Lucid TRI230c
 focal length   = 12.00 mm
 FOV   Diagonal = 36.1 degrees
     Horizontal = 30.9 degrees
       Vertical = 19.6 degrees
  GSD at 4.00 m = 1.2 mmm



## Luxonis Oak-D POE

[product website](https://docs.luxonis.com/projects/hardware/en/latest/pages/BW1098OAK.html)

[technical info](https://docs.luxonis.com/projects/hardware/en/latest/pages/SJ2088POE.html#minimal-and-maximal-perceiving-distances-of-the-camera)

Color Sensor: IMX378

Price: $299
I strongly recommend purchasing this camera, as it would solve a number of problems with the existing Zed/Lucid hybrid.

In [7]:
width_pixels = 4056
height_pixels = 3040
pitch_um = 1.55
dfov_deg = 81.0
focal_mm = compute_focal_mm(dfov_deg * np.pi / 180, width_pixels, height_pixels, pitch_um)
print_summary("Luxonis Oak-D POE", width_pixels, height_pixels, pitch_um, focal_mm, distance_m)

Luxonis Oak-D POE
 focal length   = 4.60 mm
 FOV   Diagonal = 81.0 degrees
     Horizontal = 68.7 degrees
       Vertical = 54.2 degrees
  GSD at 4.00 m = 1.3 mmm

